In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import random

In [2]:
class RemoveCorrelationTransformer2(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7):
        self.correlation_threshold = correlation_threshold


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson', min_periods=1)
        df_not_correlated = ~(df_corr.mask(
            np.tril(np.ones([len(df_corr)] * 2, dtype=bool))).abs() > self.correlation_threshold).any()
        self.un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        df = df[self.un_corr_idx]
        return df.values

In [3]:

class RemoveCorrelationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7, pca_components_ratio=3):
        self.correlation_threshold = correlation_threshold
        self.pca_components_ratio = pca_components_ratio


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson')
        df_corr = df_corr - np.eye(df.shape[1])
        outliares_corr = df_corr[np.abs(df_corr) > self.correlation_threshold]
        self.outliares_corr = outliares_corr.dropna(axis=1, how='all')

        correlated_df = df[self.outliares_corr.columns]

        n_components = len(self.outliares_corr.columns) // self.pca_components_ratio
        pca = PCA(n_components=n_components)

        correlated_df = pca.fit_transform(correlated_df)
        self.correlated_df = pd.DataFrame(correlated_df, columns=["pca_{}".format(i) for i in range(n_components)])

        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        df = df.drop((self.outliares_corr.columns), axis=1)
        df = df.join(self.correlated_df)
        return df

In [4]:

class RemoveMissingFeaturesTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, Y=None):
        self.is_missing = X.isnull().values.any(axis=0)
        return self

    def transform(self, X, Y=None):
        copy_x = pd.DataFrame(X)
        self.is_missing += copy_x.isnull().values.any(axis=0)

        copy_x = copy_x.iloc[:, ~self.is_missing]

        return copy_x.values


In [5]:
def refactor_labels(df):
    return df.replace({'low': 0 ,'high': 1, 'clinical': 1 })


def get_data(file_name, LSAS_threshold=None):
    group_column = 'group'
    sub_num_col = 'Subject_Number'
    lsas_col = 'LSAS'
    df = pd.read_excel(file_name, sheet_name='Sheet1')
    if LSAS_threshold is None:
        X = df.drop([group_column, sub_num_col, lsas_col], 1)
        Y = refactor_labels(df[group_column])
        return X, Y
    else:
        
        X = df.drop([group_column], 1)
        Y = pd.Series(np.where(X[lsas_col] > LSAS_threshold, 1, 0))
        X = X.drop([sub_num_col, lsas_col], 1)
        return X, Y


## get training data

In [6]:
file_name = "training_set_100.xlsx"

In [7]:
X_train, y_train = get_data(file_name, LSAS_threshold = 50)

## shuffle

In [8]:
random.seed(217828)

In [9]:
columns_shuffled = list(X_train.columns)

In [10]:
random.shuffle(columns_shuffled)
X_train = X_train[columns_shuffled]

In [11]:
X_train.columns

Index(['Ratio N/D+N', 'STD_fixation_length_All',
       'average_fixation_length_Neutral',
       'avg_of_sum_fixation_length_Disgusted',
       'avg_of_amount_fixation_Disgusted', 'STD_fixation_length_Disgusted',
       'avg_of_amount_fixation_White_Space', 'STD_pupil_size_Neutral',
       'average_fixation_length_Disgusted', 'avg_of_amount_fixation_Neutral',
       'average_fixation_length_White_Space', 'mean_different_AOI_per_trial',
       'average_pupil_size_All', 'STD_fixation_length_White_Space',
       'STD_pupil_size_Disgusted', 'STD_pupil_size_White_Space',
       'average_pupil_size_Disgusted', 'average_pupil_size_Neutral',
       'avg_of_sum_fixation_length_Neutral', 'Ratio D/D+N',
       'STD_fixation_length_Neutral', 'STD_pupil_size_All', 'var_ratio_D_DN',
       'average_pupil_size_White_Space',
       'avg_of_sum_fixation_length_White_Space'],
      dtype='object')

In [12]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, stratify=y_train)


## get test data

In [13]:
file_name = "full_test_set.xlsx"
df = pd.read_excel(file_name, sheet_name='Sheet1')
X_test = df.drop('Subject_Number', axis = 1)

## training pipeline

In [14]:
pipe =  Pipeline([
    ("rnf", RemoveMissingFeaturesTransformer()), 
    ('correlation_threshold', RemoveCorrelationTransformer2()), 
    ('rfc', RFE(RandomForestClassifier(n_estimators = 100))),
    ('classifier', XGBClassifier())])

In [27]:
params_grid = [
    {
        'correlation_threshold__correlation_threshold' : [0.8, 0.9],
        'rfc__n_features_to_select': [19, 22, 17, 15],
        'classifier__min_child_weight': [1],
        'classifier__gamma': [1, 2, 0.3,  0.5, 0.7],
        'classifier__subsample': [0.9],
        'classifier__colsample_bytree': [0.7],
        'classifier__max_depth': [3, 4],
        'classifier__reg_alpha' : [0.7],
        'classifier__reg_lambda' : [0.2],
        'classifier__learning_rate': [0.08, 0.05, 0.1],
        'classifier__n_estimators': [65, 95]}] 

## grid search

In [28]:
cv = StratifiedKFold(10)
gs = GridSearchCV(pipe, params_grid, cv=cv, scoring='accuracy')
gs.fit(X_train_2, y_train_2)

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('rnf', RemoveMissingFeaturesTransformer()), ('correlation_threshold', RemoveCorrelationTransformer2(correlation_threshold=0.7)), ('rfc', RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=N...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'correlation_threshold__correlation_threshold': [0.8, 0.9], 'rfc__n_features_to_select': [19, 22, 17, 15], 'classifier__min_child_weight': [1], 'classifier__gamma': [1, 2, 0.3, 0.5, 0.7], 'classifier__subsample': [0.9], 'classifier__colsample_bytree': [0.7], 'classifier__max_depth': [3, 4], 'classifier__reg_alpha': [0.7], 'classifier__reg_lambda': [0.2], 'cla

In [29]:
gs.best_params_

{'classifier__colsample_bytree': 0.7,
 'classifier__gamma': 1,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 3,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 95,
 'classifier__reg_alpha': 0.7,
 'classifier__reg_lambda': 0.2,
 'classifier__subsample': 0.9,
 'correlation_threshold__correlation_threshold': 0.8,
 'rfc__n_features_to_select': 19}

In [30]:
gs.best_score_


0.6555555555555556

## validate grid search score  LOO


In [31]:
results = []
k = 100
for i in range(k):
    loo = LeaveOneOut()
    score = cross_val_score(gs.best_estimator_, X_train_2, y_train_2, cv=loo)
    results.append(score.mean())
sum(results)/k

KeyboardInterrupt: 

## validate grid search score  10 folds

In [ ]:
results = []
k = 100
for i in range(k):
    cv = StratifiedKFold(10)
    score = cross_val_score(gs.best_estimator_, X_train_2, y_train_2, cv=cv)
    results.append(score.mean())
sum(results)/k

## holdout

In [ ]:
model = gs.best_estimator_.fit(X_train_2, y_train_2)

In [ ]:
y_pred = model.predict(X_test_2)

In [ ]:
accuracy_score(y_pred, y_test_2)

## scores on full set LOO

In [ ]:
results = []
k = 100
for i in range(k):
    loo = LeaveOneOut()
    score = cross_val_score(gs.best_estimator_, X_train, y_train, cv=loo)
    results.append(score.mean())
sum(results)/k

## scores on full set 10 folds

In [ ]:
results = []
k = 100
for i in range(k):
    cv = StratifiedKFold(10)
    score = cross_val_score(gs.best_estimator_, X_train, y_train, cv=cv)
    results.append(score.mean())
sum(results)/k